## GSC+ DataSet translation with GoogleTranslate

In [ ]:
#Install GoogleTrans module
!pip3 install googletrans

In [34]:
import pandas as pd
import requests
import time
import googletrans
from googletrans import Translator
import json

In [35]:
gsc_data = pd.read_csv('GSC+_Es.csv')

In [36]:
gsc_data.head(10)

,file,spanish,HPO_symptoms,HPO_code
0,1003450,En cinco miembros de una familia italiana se r...,"['brachydactyly', 'ausencia de algunas falange...","['0001156', '0009881', '0001798', '0001792', '..."
1,10051003,El síndrome de Townes-Brock (TBS) es un trasto...,"['trastorno autosómico dominante.', 'autosómic...","['0000006', '0000006', '0000006', '0003828', '..."


In [37]:
def google_translate(source, target, text):
    translator = Translator()
    result = translator.translate(text, src=source, dest=target)
    return result.text

In [38]:
def translate_doc(source_language, target_language, doc_id,text, symptoms, hpo_codes):
    
    #Translate the description of the condition (symptom(s))
    translation = google_translate(source_language,target_language,text)
    translated_symptoms = []
    before_symp = []
    before_symp.append(symptoms)
    
    #translate the symtoms themselves
    for symptom in before_symp:
        t_symptom = google_translate(source_language,target_language,symptom)
        translated_symptoms.append(t_symptom)
        
    #Return all translated data         
    return {
            'file': doc_id,
            'text': translation,
            'HPO_symptoms': translated_symptoms[0],
            'HPO_code': hpo_codes
        }

In [39]:
#Translate all rows and create a dataframe

translation_rows = []
src_lang = 'es'
tgt_lang = 'en'

for row in gsc_data.itertuples():
    translation_output = translate_doc(src_lang,tgt_lang, str(row[1]), row[2], row[3], row[4])
    translation_rows.append(translation_output)
    if (row[0] + 1) % 10 == 0:
        print('Fetched {} rows'.format(row[0] + 1))
        time.sleep(1)
translation_df = pd.DataFrame(translation_rows)

In [40]:
#Drop all Nan values
translation_df.head(10)

,file,text,HPO_symptoms,HPO_code
0,1003450,Brachydactyly syndrome (absence of some middle...,"['brachydactyly', 'absence of some middle or d...","['0001156', '0009881', '0001798', '0001792', '..."
1,10051003,Townes-Brock syndrome (TBS) is an autosomal do...,"['autosomal dominant disorder.', 'autosomal do...","['0000006', '0000006', '0000006', '0003828', '..."


In [41]:
#Save to file
translation_df.to_csv('translated_gsc_es-en.csv')